In [2]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv('perceel.csv') 
# Preview the first 5 lines of the loaded data 
data.columns =['perceelid', 'perceeltype', 'huisnr', 'huisnr_bag_letter','huisnr_bag_toevoeging','bag_nummeraanduidingid','bag_adresseerbaarobjectid','lat','lon','rdx','rdy','oppervlakte','gebruiksdoel','reeksid'] 
data.head()

/Users/jellevanderschoot/.pyenv/versions/3.9.4/envs/venv_python_dsp/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,perceelid,perceeltype,huisnr,huisnr_bag_letter,huisnr_bag_toevoeging,bag_nummeraanduidingid,bag_adresseerbaarobjectid,lat,lon,rdx,rdy,oppervlakte,gebruiksdoel,reeksid
0,2,verblijfsobject,105,NaN,2,3.632000e+14,3.630100e+14,52.377782,4.905169,122177.0,487877.0,96,kantoorfunctie,1
1,3,verblijfsobject,105,NaN,3,3.632000e+14,3.630100e+14,52.377782,4.905169,122177.0,487877.0,1,woonfunctie,1
2,4,verblijfsobject,105,NaN,H,3.632000e+14,3.630100e+14,52.377782,4.905169,122177.0,487877.0,1,winkelfunctie,1
3,5,verblijfsobject,106,NaN,1,3.632000e+14,3.630100e+14,52.377835,4.905286,122185.0,487882.9,1,kantoorfunctie,2
4,6,verblijfsobject,106,NaN,2,3.632000e+14,3.630100e+14,52.377836,4.905286,122185.0,487883.0,91,kantoorfunctie,2


In [5]:
### Generate the subset for Amsterdam of perceel.csv
### Don't run this again, the file is in the folder and is called amsterdam.csv

#Links onder: 117623, 483348
#Rechts onder: 122646, 483714
#Links boven: 117761, 485643
#Rechts boven: 122155, 485407

#rdx_min = 117623
#rdx_max = 122646
#rdy_min = 483348
#rdy_max = 485407

rdx_min = 121623
rdx_max = 122146
rdy_min = 484348
rdy_max = 484907

df_kadam = data[(data['rdx'] > rdx_min) & 
               (data['rdx'] < rdx_max) &
              (data['rdy'] > rdy_min) &
              (data['rdy'] < rdy_max)]

df_kadam.to_csv('amsterdamzuid-klein.csv')

In [18]:
### Converter from lat/lon to rdx/rdy

from builtins import enumerate

class RDWGS84Converter(object):

    x0 = 155000
    y0 = 463000
    phi0 = 52.15517440
    lam0 = 5.38720621

    # Coefficients or the conversion from RD to WGS84
    Kp = [0, 2, 0, 2, 0, 2, 1, 4, 2, 4, 1]
    Kq = [1, 0, 2, 1, 3, 2, 0, 0, 3, 1, 1]
    Kpq = [3235.65389, -32.58297, -0.24750, -0.84978, -0.06550, -0.01709, -0.00738, 0.00530, -0.00039,
           0.00033, -0.00012]

    Lp = [1, 1, 1, 3, 1, 3, 0, 3, 1, 0, 2, 5]
    Lq = [0, 1, 2, 0, 3, 1, 1, 2, 4, 2, 0, 0]
    Lpq = [5260.52916, 105.94684, 2.45656, -0.81885, 0.05594, -0.05607, 0.01199, -0.00256, 0.00128, 0.00022,
           -0.00022, 0.00026]
    # Coefficients for the conversion from WGS84 to RD
    Rp = [0, 1, 2, 0, 1, 3, 1, 0, 2]
    Rq = [1, 1, 1, 3, 0, 1, 3, 2, 3]
    Rpq = [190094.945, -11832.228, -114.221, -32.391, -0.705, -2.340, -0.608, -0.008, 0.148]

    Sp = [1, 0, 2, 1, 3, 0, 2, 1, 0, 1]
    Sq = [0, 2, 0, 2, 0, 1, 2, 1, 4, 4]
    Spq = [309056.544, 3638.893, 73.077, -157.984, 59.788, 0.433, -6.439, -0.032, 0.092, -0.054]

    def from_rd(self, x: int, y: int) -> tuple:
        """
        Converts RD coordinates into WGS84 coordinates
        """
        dx = 1E-5 * (x - self.x0)
        dy = 1E-5 * (y - self.y0)
        latitude = self.phi0 + sum([v * dx ** self.Kp[i] * dy ** self.Kq[i] for i, v in enumerate(self.Kpq)]) / 3600
        longitude = self.lam0 + sum([v * dx ** self.Lp[i] * dy ** self.Lq[i] for i, v in enumerate(self.Lpq)]) / 3600

        return latitude, longitude

    def from_wgs84(self, latitude: float, longitude: float) -> tuple:
        """
        Converts WGS84 coordinates into RD coordinates
        """
        dlat = 0.36 * (latitude - self.phi0)
        dlon = 0.36 * (longitude - self.lam0)
        x = self.x0 + sum([v * dlat ** self.Rp[i] * dlon ** self.Rq[i] for i, v in enumerate(self.Rpq)])
        y = self.y0 + sum([v * dlat ** self.Sp[i] * dlon ** self.Sq[i] for i, v in enumerate(self.Spq)])

        return x, y

In [19]:
### Old material

straal = 2000
wgs84 = (52.692644, 4.747290)

rdx_punt,rdy_punt = from_wgs84(RDWGS84Converter,wgs84[0],wgs84[1])
print(rdx_punt)
print(rdy_punt)

rdx_min = rdx_punt - straal
rdx_max = rdx_punt + straal

rdy_min = rdy_punt - straal
rdy_max = rdy_punt + straal

rslt_df = data[(data['rdx'] > rdx_min) & 
               (data['rdx'] < rdx_max) &
              (data['rdy'] > rdy_min) &
              (data['rdy'] < rdy_max)]
#(data['gebruiksdoel'] == 'woonfunctie')
print(str(len(rslt_df.index)) + " addressen")
display(rslt_df)

NameError: name 'from_wgs84' is not defined

In [56]:
rslt_df['gebruiksdoel'] == 'winkelfunctie'


print((rslt_df['gebruiksdoel']=='woonfunctie').sum())
print((rslt_df['gebruiksdoel']!='woonfunctie').sum())

27868
0


In [1]:
import tkinter as tk
from builtins import enumerate

import pandas as pd 
data = pd.read_csv('perceel.csv') 
data.columns =['perceelid', 'perceeltype', 'huisnr', 'huisnr_bag_letter','huisnr_bag_toevoeging','bag_nummeraanduidingid','bag_adresseerbaarobjectid','lat','lon','rdx','rdy','oppervlakte','gebruiksdoel','reeksid'] 
data.head()

class RDWGS84Converter(object):

    x0 = 155000
    y0 = 463000
    phi0 = 52.15517440
    lam0 = 5.38720621

    # Coefficients or the conversion from RD to WGS84
    Kp = [0, 2, 0, 2, 0, 2, 1, 4, 2, 4, 1]
    Kq = [1, 0, 2, 1, 3, 2, 0, 0, 3, 1, 1]
    Kpq = [3235.65389, -32.58297, -0.24750, -0.84978, -0.06550, -0.01709, -0.00738, 0.00530, -0.00039,
           0.00033, -0.00012]

    Lp = [1, 1, 1, 3, 1, 3, 0, 3, 1, 0, 2, 5]
    Lq = [0, 1, 2, 0, 3, 1, 1, 2, 4, 2, 0, 0]
    Lpq = [5260.52916, 105.94684, 2.45656, -0.81885, 0.05594, -0.05607, 0.01199, -0.00256, 0.00128, 0.00022,
           -0.00022, 0.00026]
    # Coefficients for the conversion from WGS84 to RD
    Rp = [0, 1, 2, 0, 1, 3, 1, 0, 2]
    Rq = [1, 1, 1, 3, 0, 1, 3, 2, 3]
    Rpq = [190094.945, -11832.228, -114.221, -32.391, -0.705, -2.340, -0.608, -0.008, 0.148]

    Sp = [1, 0, 2, 1, 3, 0, 2, 1, 0, 1]
    Sq = [0, 2, 0, 2, 0, 1, 2, 1, 4, 4]
    Spq = [309056.544, 3638.893, 73.077, -157.984, 59.788, 0.433, -6.439, -0.032, 0.092, -0.054]

    def from_wgs84(self, latitude: float, longitude: float) -> tuple:
        """
        Converts WGS84 coordinates into RD coordinates
        """
        dlat = 0.36 * (latitude - self.phi0)
        dlon = 0.36 * (longitude - self.lam0)
        x = self.x0 + sum([v * dlat ** self.Rp[i] * dlon ** self.Rq[i] for i, v in enumerate(self.Rpq)])
        y = self.y0 + sum([v * dlat ** self.Sp[i] * dlon ** self.Sq[i] for i, v in enumerate(self.Spq)])

        return x, y

fields = ('r', 'lat', 'lon')

def rk(entries):
    lat = float(entries['lat'].get())
    lon = float(entries['lon'].get())
    straal = (float(entries['r'].get()))
    
    rdx_punt,rdy_punt = RDWGS84Converter.from_wgs84(RDWGS84Converter,lat,lon)
    rdx_min = rdx_punt - straal
    rdx_max = rdx_punt + straal

    rdy_min = rdy_punt - straal
    rdy_max = rdy_punt + straal

    rslt_df = data[(data['rdx'] > rdx_min) & 
                   (data['rdx'] < rdx_max) &
                  (data['rdy'] > rdy_min) &
                  (data['rdy'] < rdy_max)]
    print(str(len(rslt_df.index)) + " addressen")

def makeform(root, fields):
    entries = {}
    for field in fields:
        print(field)
        row = tk.Frame(root)
        lab = tk.Label(row, width=22, text=field+": ", anchor='w')
        ent = tk.Entry(row)
        ent.insert(0, "0")
        row.pack(side=tk.TOP, 
                 fill=tk.X, 
                 padx=5, 
                 pady=5)
        lab.pack(side=tk.LEFT)
        ent.pack(side=tk.RIGHT, 
                 expand=tk.YES, 
                 fill=tk.X)
        entries[field] = ent
    return entries

if __name__ == '__main__':
    root = tk.Tk()
    ents = makeform(root, fields)
    b1 = tk.Button(root, text='Bereken adressen',
           command=(lambda e=ents: rk(e)))
    b1.pack(side=tk.LEFT, padx=5, pady=5)
    b2 = tk.Button(root, text='Quit', command=root.quit)
    b2.pack(side=tk.LEFT, padx=5, pady=5)
    root.mainloop()

r
lat
lon
0 addressen
